## Bibliotecas

In [44]:
import pandas as pd
import numpy as np
import inflection #função snakecase
import math

## Entendendo os dados originais da Rossmann


##### 0. store:
- um ID exclusivo para cada loja

##### 1. day_of_week:
- dia da semana em que a venda ocorreu (dados não descritos)

##### 2. date:
- data de venda (dados não descritos)

##### 3. sales:
- o volume de negócios para um determinado dia (isto é o que você está prevendo) 

##### 4. customers
- o número de clientes em um determinado dia  

##### 5. open: 
- um indicador para saber se a loja estava aberta: 0 = fechado, 1 = aberto  

##### 6.  promo:
- indica se uma loja está realizando uma promoção naquele dia

##### 7. state_holiday:
- indica um feriado estadual. Normalmente todas as lojas, com poucas exceções, fecham nos feriados estaduais. Observe que todas as escolas fecham nos feriados e fins de semana. a = feriado, b = feriado da Páscoa, c = Natal, 0 = nenhum

##### 8. school_holiday:
- indica se o (store, date) foi afetado pelo fechamento das escolas públicas

##### 9. store_type:
- diferencia entre 4 modelos de loja diferentes: a, b, c, d

##### 10. assortment: 
- descreve um nível de sortimento: a = básico, b = extra, c = estendido

##### 11. competition_distance:
- distância em metros até a loja concorrente mais próxima

##### 12. competition_open_since_month:
- fornece o mês aproximado em que o concorrente mais próximo foi aberto

##### 13. competition_open_since_year:
- fornece o ano aproximado em que o concorrente mais próximo foi aberto

##### 14 . promo2:
- promo2 é uma promoção contínua e consecutiva para algumas lojas: 0 = a loja não está participando, 1 = a loja está participando

##### 15. promo2_since_week:
- descreve a semana em que a loja começou a participar da 'promo2'

##### 16. promo2_since_year
- descreve o ano em que a loja começou a participar da 'promo2'

##### 17.  promo_interval:
- meses em que a promo2 ficou ativa

In [45]:
#obs: coluna 'id' se encontra na descrição, porém não no dataset

## Carregamento dos dados

In [46]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# merge (unindo os arquivos csv)
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

## Funções adicionais

## Descrição dos dados

### 1. Renomeando colunas

In [47]:
#criando uma cópia do dataset para trabalhar com os dados
df1 = df_raw.copy()

In [48]:
# nomes das colunas antigas
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [49]:
cols_originais = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday',
                  'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
                  'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']


# criar função para modificar os nomes
def snakecase(x):
    return inflection.underscore(x)


# salvar colunas novas e aplicar função anterior utilizando o map
cols_novas = list(map(snakecase, cols_originais))

# renomear
df1.columns = cols_novas

In [50]:
# checar modificação nos nomes das colunas

df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

### 2. Verificando a dimensão dos dados

In [51]:
print(f'Linhas: {df1.shape[0]}')
print(f'Colunas: {df1.shape[1]}')

Linhas: 1017209
Colunas: 18


### 3. Verificando os tipos de dados

In [52]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [53]:
# coluna 'date' de object para datetime
df1['date'] = pd.to_datetime(df1['date'])

In [54]:
# checando modificações
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

### 4. Checando dados nulos (NA)

In [55]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

### 5. Substituindo as colunas que possuem dados nulos

##### comptetition_distance:
- distância em metros do competidor mais próximo
- sugestão: substituir os dados nulos por um número acima do valor máximo de distância, pois ou não há competidor ou está longe o suficiente para não ser considerado um

##### competition_open_since_month:
- mês que o competidor mais próximo foi aberto
- sugestão: substituir os dados nulos do mês de abertura pelo mês da data de venda registrada (coluna 'date') na linha

##### competition_open_since_year:
- ano que o competidor mais pŕoximo foi aberto
- sugestão: substituir os dados nulos do ano de abertura pelo ano da data de venda registrada (coluna 'date') na linha

##### OBSERVAÇÃO:
- coluna promo2: significa continuação de uma promoção, 0 -  não está participando; 1- está participando
- as colunas seguintes dependem dessa
- importante: sempre onde a promo2 é 0, os dados seguintes são nulos, ou seja, apenas significam que não houve promo2

##### promo2_since_week:
- semana em que a loja começou a participar da promo2
- sugestão: substituir os dados nulos extraindo a semana da data de venda registrada (coluna 'date')

##### promo2_since_year:
- ano em que a loja começou a participar da promo2
- sugestão:substituir os dados nulos extraindo o ano da data de venda registrada (coluna 'date')

##### promo_interval:
- meses em que a promo2 ficou ativa

- sugestão 1: substituir dados nulos da 'promo_interval' por 0

- sugestão 2 ['month_map']: adicionar meses na coluna nova 'month_map' de acordo com o mês presente na coluna de data de venda 'date'

- sugestão 3 ['is_promo']: se o mês 'month_map' estiver dentro da lista presente em 'promo_interval', adicionar: 1-tem promoção; 0-não tem promoção.

In [56]:
# encontrar o valor máximo para utilizar de parâmetro para a 'competition_distance'
df1['competition_distance'].max()

75860.0

In [57]:
# competition_distance

df1['competition_distance'] = df1['competition_distance'].apply(
    lambda x: 200000.0 if math.isnan(x) else x)

In [58]:
# competition_open_since_month

df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(
    x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

In [59]:
# competition_open_since_year

df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(
    x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

In [60]:
# promo2_since_week

df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(
    x['promo2_since_week']) else x['promo2_since_week'], axis=1)

In [61]:
# promo2_since_year

df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(
    x['promo2_since_year']) else x['promo2_since_year'], axis=1)

In [76]:
# promo_interval

#criar dicionario com os meses
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
             7: 'Jul', 8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

#sugestão 1: se for nan, vai substituir por 0
df1['promo_interval'].fillna(0, inplace=True)

#sugestão 2: ['month_map']
df1['month_map'] = df1['date'].dt.month.map(month_map)

#sugestão 3: ['is_promo']
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(
    lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [64]:
# checando o tratamento de NA
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

### 6. Etapa 

### 7. Etapa 

### 8. Etapa 

### +